In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
from descriptastorus.descriptors import rdNormalizedDescriptors
from rdkit import Chem
import logging

# make the normalized descriptor generator
generator = rdNormalizedDescriptors.RDKit2DNormalized()
generator.columns # list of tuples:  (descriptor_name, numpytype) ...

# features = generator.process(smiles)
# features[0] is True/False if the smiles could be processed correcty
# features[1:] are the normalized descriptors as described in generator.columns

# example for converting a smiles string into the values
def rdkit_2d_normalized_features(smiles: str):
    # n.b. the first element is true/false if the descriptors were properly computed
    results = generator.process(smiles)
    processed, features = results[0], results[1:]
    if processed is None:
       logging.warning("Unable to process smiles %s", smiles)
    # if processed is None, the features are are default values for the type
    return features

In [3]:
ERROR_COLUMN = 'Absolute Error'
SMILES_COLUMN = 'smiles'
VALUE_COLUMN = 'logP'
PREDS_COLUMN = 'logP_pred'

DATASET_INPUT_PATH = '../../../data/3_final_data/split_data'

DATASET_OUTPUT_PATH = '../../../data/raw/baselines/dmpnn'

In [4]:
train_data = pd.read_csv(os.path.join(DATASET_INPUT_PATH, 'logp_wo_averaging_train.csv'))
val_data = pd.read_csv(os.path.join(DATASET_INPUT_PATH, 'logp_wo_averaging_validation.csv'))
test_data = pd.read_csv(os.path.join(DATASET_INPUT_PATH, 'logp_wo_averaging_test.csv'))

In [5]:
def create_feature_dataframe(df):
    import numpy as np
    rdkit_table = []
    features_names = [gen[0] for gen in generator.columns]
    smiles_index_dict = {}
    for i in tqdm(range(df.shape[0])):
        smiles = df.iloc[i][SMILES_COLUMN]
        logP = df.iloc[i][VALUE_COLUMN]
        features = {features_names[j]:feature for j,feature in enumerate(rdkit_2d_normalized_features(smiles))}
        features[SMILES_COLUMN] = smiles
        features[VALUE_COLUMN] = logP
        rdkit_table.append(features)
        smiles_index_dict[smiles]=i
    rdkit_features = pd.DataFrame(rdkit_table)
    return rdkit_features, smiles_index_dict

In [6]:
train_data_rdkit, train_smiles_dict = create_feature_dataframe(train_data)
val_data_rdkit, val_smiles_dict = create_feature_dataframe(val_data)
test_data_rdkit, test_smiles_dict = create_feature_dataframe(test_data)

100%|██████████| 2067/2067 [04:27<00:00,  7.73it/s]


In [7]:
import json

train_data_rdkit.to_csv(os.path.join(DATASET_OUTPUT_PATH,'logp_wo_averaging_train_drkit_feat.csv'))
with open(os.path.join(DATASET_OUTPUT_PATH,'logp_wo_averaging_train_smiles_dict.json'), 'w') as f:
    json.dump(train_smiles_dict, f)
val_data_rdkit.to_csv(os.path.join(DATASET_OUTPUT_PATH,'logp_wo_averaging_val_drkit_feat.csv'))
with open(os.path.join(DATASET_OUTPUT_PATH,'logp_wo_averaging_val_smiles_dict.json'), 'w') as f:
    json.dump(val_smiles_dict, f)
test_data_rdkit.to_csv(os.path.join(DATASET_OUTPUT_PATH,'logp_wo_averaging_test_drkit_feat.csv'))
with open(os.path.join(DATASET_OUTPUT_PATH,'logp_wo_averaging_test_smiles_dict.json'), 'w') as f:
    json.dump(test_smiles_dict, f)


# Train model

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
VALUE_COLUMN = 'logP'
SMILES_COLUMN = 'smiles'

DATA_PATH = '../../../data/raw/baselines/dmpnn'

In [3]:
train_data = pd.read_csv(os.path.join(DATA_PATH, 'logp_wo_averaging_train_drkit_feat.csv'), index_col=0)
val_data = pd.read_csv(os.path.join(DATA_PATH, 'logp_wo_averaging_val_drkit_feat.csv'), index_col=0)
test_data = pd.read_csv(os.path.join(DATA_PATH, 'logp_wo_averaging_test_drkit_feat.csv'), index_col=0)

In [11]:
y_train = train_data[VALUE_COLUMN]
X_train = train_data.drop([VALUE_COLUMN, SMILES_COLUMN], axis = 1)
X_train = X_train.fillna(0)

y_val = val_data[VALUE_COLUMN]
X_val = val_data.drop([VALUE_COLUMN, SMILES_COLUMN], axis = 1)
X_val = X_val.fillna(0)


y_test = test_data[VALUE_COLUMN]
X_test = test_data.drop([VALUE_COLUMN, SMILES_COLUMN], axis = 1)
X_test = X_test.fillna(0)

## XGBoost

In [18]:
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [19]:
model = XGBRegressor(n_estimators = 100, max_depth = 6, random_state = 42)

In [20]:
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
test_preds = model.predict(X_test)
val_preds = model.predict(X_val)

In [22]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds, squared=False))
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds, squared=False))
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

Valid RMSE = 0.550378750673703
Valid R2-score is 0.9101585772785692
Test RMSE = 0.5765899578624547
Test R2-score is 0.9012773315417384


In [23]:
feature_importance = model.feature_importances_

In [24]:
features = list(X_train)

In [25]:
df = pd.DataFrame(columns = ['feature importance', 'feature name'])

In [26]:
df['feature importance'] = feature_importance
df['feature name'] = features
df_20_most_important = df.sort_values(by='feature importance', ascending = False)[:20]
with open(os.path.join(DATA_PATH,'experiments_result'),'w') as f:
    f.write(df_20_most_important.reset_index(drop=True).to_markdown())

# MLP

In [27]:
scaler = StandardScaler()

In [28]:
X_train_scaled = scaler.fit_transform(X_train)

In [29]:
regr = MLPRegressor(random_state=10, max_iter=100, warm_start=True, early_stopping = True, solver = 'lbfgs')

In [30]:
regr.fit(X_train_scaled, y_train)

/Users/Alisa.Alenicheva/anaconda3/envs/chemprop/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPRegressor(early_stopping=True, max_iter=100, random_state=10, solver='lbfgs',
             warm_start=True)

In [31]:
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val) 

In [32]:
test_preds = regr.predict(X_test_scaled)
val_preds = regr.predict(X_val_scaled)

In [33]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds, squared=False))
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds, squared=False))
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))

Valid RMSE = 0.5670377404502589
Valid R2-score is 0.9046375862326349
Test RMSE = 0.5642646297928993
Test R2-score is 0.9054528606765462


## XGBoost without MolLogP

In [4]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [5]:
VALUE_COLUMN = 'logP'
SMILES_COLUMN = 'smiles'

DATA_PATH = '../../../data/raw/baselines/dmpnn'

In [6]:
train_data = pd.read_csv(os.path.join(DATA_PATH, 'logp_wo_averaging_train_drkit_feat.csv'), index_col=0)
val_data = pd.read_csv(os.path.join(DATA_PATH, 'logp_wo_averaging_val_drkit_feat.csv'), index_col=0)
test_data = pd.read_csv(os.path.join(DATA_PATH, 'logp_wo_averaging_test_drkit_feat.csv'), index_col=0)

In [7]:
y_train = train_data[VALUE_COLUMN]
X_train = train_data.drop([VALUE_COLUMN, SMILES_COLUMN, 'MolLogP'], axis = 1)
X_train = X_train.fillna(0)

y_val = val_data[VALUE_COLUMN]
X_val = val_data.drop([VALUE_COLUMN, SMILES_COLUMN, 'MolLogP'], axis = 1)
X_val = X_val.fillna(0)


y_test = test_data[VALUE_COLUMN]
X_test = test_data.drop([VALUE_COLUMN, SMILES_COLUMN, 'MolLogP'], axis = 1)
X_test = X_test.fillna(0)

In [8]:
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [6]:
model = XGBRegressor(n_estimators = 100, max_depth = 6, random_state = 42)

In [7]:
model.fit(X_train, y_train)

/home/mol/anaconda3/envs/jtree/lib/python2.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/mol/anaconda3/envs/jtree/lib/python2.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [8]:
test_preds = model.predict(X_test)
val_preds = model.predict(X_val)

In [9]:
print("Valid RMSE =", mean_squared_error(y_val, val_preds)**0.5)
print("Valid R2-score is {0}".format(r2_score(y_val, val_preds)))

print("Test RMSE =", mean_squared_error(y_test, test_preds)**0.5)
print("Test R2-score is {0}".format(r2_score(y_test, test_preds)))


('Valid RMSE =', 0.5891643504760782)
Valid R2-score is 0.897050029756
('Test RMSE =', 0.5829575142928344)
Test R2-score is 0.89908480897


In [10]:
feature_importance = model.feature_importances_

In [11]:
features = list(X_train)

In [12]:
df = pd.DataFrame(columns = ['feature importance', 'feature name'])

In [13]:
df['feature importance'] = feature_importance
df['feature name'] = features
df_20_most_important = df.sort_values(by='feature importance', ascending = False)[:20]
# with open(os.path.join(DATA_PATH,'experiments_result'),'w') as f:
#     f.write(df_20_most_important.reset_index(drop=True).to_markdown())

In [17]:
for i in range(len(df_20_most_important)):
    print '|', i, '|', df_20_most_important.iloc[i]['feature importance'], '|', df_20_most_important.iloc[i]['feature name'], '|'

| 0 | 0.13547863 | PEOE_VSA6 |
| 1 | 0.08802548 | fr_benzene |
| 2 | 0.056294214 | NHOHCount |
| 3 | 0.04401501 | PEOE_VSA7 |
| 4 | 0.043028124 | fr_alkyl_halide |
| 5 | 0.03891857 | NumHDonors |
| 6 | 0.0365785 | NumAromaticCarbocycles |
| 7 | 0.022730295 | fr_Al_OH_noTert |
| 8 | 0.019696752 | LabuteASA |
| 9 | 0.015776157 | fr_halogen |
| 10 | 0.015054731 | RingCount |
| 11 | 0.014066192 | fr_Al_COO |
| 12 | 0.013386708 | MolWt |
| 13 | 0.010992395 | NumAliphaticCarbocycles |
| 14 | 0.010782038 | VSA_EState7 |
| 15 | 0.010725621 | VSA_EState10 |
| 16 | 0.010548814 | NumAromaticRings |
| 17 | 0.009862682 | SMR_VSA9 |
| 18 | 0.008671275 | SMR_VSA7 |
| 19 | 0.008569112 | Chi0v |


## Grid Search

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
xgb1 = XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 10, 20, 50],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100, 200, 500, 1000]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(np.concatenate((X_train, X_val), axis=0),
         np.concatenate((y_train, y_val), axis=0))

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 48 candidates, totalling 96 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
